In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt

In [4]:
df = pd.read_csv("./res/insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [5]:
df.shape

(28, 3)

In [6]:
inputs = df.drop(['bought_insurance'], axis='columns')
targets = df.bought_insurance

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.2)

In [8]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age']/100
X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age']/100

In [9]:
model = keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.fit(X_train_scaled,y_train, epochs=5000)

/home/javedjarif/Desktop/workdir/Tensorflow_Keras/env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-03-29 23:22:54.025592: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 855ms/step - accuracy: 0.4545 - loss: 0.7623
Epoch 2/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4545 - loss: 0.7618
Epoch 3/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.4545 - loss: 0.7613
Epoch 4/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4545 - loss: 0.7609
Epoch 5/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4545 - loss: 0.7604
Epoch 6/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.4545 - loss: 0.7599
Epoch 7/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4545 - loss: 0.7595
Epoch 8/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4545 - loss: 0.7590
Epoch 9/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.4545 - loss: 0.7586
Epoch 10/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4545 - loss: 0.7581
Epoch 11/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.4545 - loss: 0.7576
Epoch 12/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy

In [10]:
model.evaluate(X_test_scaled,y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - accuracy: 0.8333 - loss: 0.4177


[0.41767433285713196, 0.8333333134651184]

In [11]:
model.predict(X_test_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


array([[0.36097962],
       [0.6938424 ],
       [0.6601096 ],
       [0.73574346],
       [0.52617574],
       [0.799655  ]], dtype=float32)

In [12]:
y_test

26    0
24    1
2     1
25    1
6     0
9     1
Name: bought_insurance, dtype: int64

In [14]:
coef, intercept = model.get_weights()
coef,intercept

(array([[5.1454053 ],
        [0.97061634]], dtype=float32),
 array([-2.7251742], dtype=float32))

In [15]:
import math
def sigmoid(x):
    return 1/(1+math.exp(-x))


sigmoid(18)

0.9999999847700205

In [16]:
X_test

,age,affordibility
26,23,1
24,50,1
2,47,1
25,54,1
6,55,0
9,61,1


In [21]:
def prediction_function(age, affordibility):
    weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
    return sigmoid(weighted_sum[0])

prediction_function(0.47, 1)

0.6601095837223907

In [22]:
prediction_function(.18, 1)

0.3039866878652925

In [23]:
def sigmoid_numpy(X):
    return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([12, 0, 1]))

array([0.99999386, 0.5       , 0.73105858])

In [24]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i, epsilon) for i in y_predicted]
    y_predicted_new = [min(i, 1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)

    return -np.mean(y_true*np.log(y_predicted_new) + (1-y_true)*np.log(1-y_predicted_new))

In [30]:
def gradient_descent(age, affordability, y_true, epochs, loss_threshold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1*age + w2*affordability + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true, y_predicted)

        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
        w2d = (1/n)*np.dot(np.transpose(affordability), (y_predicted-y_true))

        bias_d = np.mean(y_predicted-y_true)

        w1 = w1-rate*w1d
        w2 = w2-rate*w2d

        bias = bias-rate*bias_d

        print(f"Epoch:{i} w1:{w1} w2:{w2} bias:{bias} loss:{loss}")

        if(loss<=loss_threshold):
            break

    
    return w1, w2, bias

In [31]:
gradient_descent(X_train_scaled['age'],X_train_scaled['affordibility'],y_train, 1000, 0.4177)

Epoch:0 w1:0.9732428767734687 w2:0.9267886236426337 bias:-0.13339864958813177 loss:0.7622736667721061
Epoch:1 w1:0.9531467518762704 w2:0.8652350817565387 bias:-0.2488151786824383 loss:0.7179074718858922
Epoch:2 w1:0.9392193649427742 w2:0.8147219783585413 bias:-0.34769335175761745 loss:0.6855604850559059
Epoch:3 w1:0.9307962363072643 w2:0.7742107482309576 bias:-0.43188852478537687 loss:0.6625321182172815
Epoch:4 w1:0.9271404566733876 w2:0.7424465066231142 bias:-0.503407319715093 loss:0.6463834169400178
Epoch:5 w1:0.9275211478989683 w2:0.7181285668181047 bias:-0.5642094659258114 loss:0.6351225082432534
Epoch:6 w1:0.9312628915480913 w2:0.7000237873059902 bias:-0.6160872655963598 loss:0.6272362657506627
Epoch:7 w1:0.9377698714866051 w2:0.6870269805993932 bias:-0.6606101991284071 loss:0.6216329629695633
Epoch:8 w1:0.9465325278258981 w2:0.6781833036803316 bias:-0.6991133585361665 loss:0.6175528448221375
Epoch:9 w1:0.9571240233509537 w2:0.6726876427983981 bias:-0.7327106677359096 loss:0.61448

(6.801554733907704, 1.012726513518029, -3.4122164851732526)

In [32]:
coef, intercept

(array([[5.1454053 ],
        [0.97061634]], dtype=float32),
 array([-2.7251742], dtype=float32))